In [1]:
# Import necessary packages
import cv2
import itertools
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import sys
from os import listdir

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_validate

In [2]:
# Function to load the images given the folder name
def load_folder(folder, num_samples):
    imgs = []
    files = listdir('./MIO-TCD-Classification/train/{}/'.format(folder))
    img_counter = 0
    for img in files:
        # Read the images in grayscale
        img = cv2.cvtColor(cv2.imread('./MIO-TCD-Classification/train/{}/{}'.format(folder,img)), cv2.COLOR_BGR2GRAY)
        # Perform a box blur on the images
        kernel = np.array([[1/9,1/9,1/9], [1/9,1/9,1/9], [1/9,1/9,1/9]])
        img = cv2.filter2D(img, -1, kernel)
        # Resize to reduce computation time
        img = cv2.resize(img, (64,64))
        imgs.append(img)
        img_counter += 1
        if(img_counter==num_samples): break
    print('Number of samples from ' + folder + '=' + str(len(imgs)))
    return imgs

In [3]:
# The number of images from each class to import
num_samples = 10000
# Import images from each folder
articulated_truck = load_folder('articulated_truck', num_samples)
bicycle = load_folder('bicycle', num_samples)
bus = load_folder('bus', num_samples)
car = load_folder('car', num_samples)
motorcycle = load_folder('motorcycle', num_samples)
non_motorized_vehicle = load_folder('non-motorized_vehicle', num_samples)
pedestrian = load_folder('pedestrian', num_samples)
pickup_truck = load_folder('pickup_truck', num_samples)
single_unit_truck = load_folder('single_unit_truck', num_samples)
work_van = load_folder('work_van', num_samples)

Number of samples from articulated_truck=10000
Number of samples from bicycle=2284
Number of samples from bus=10000
Number of samples from car=10000
Number of samples from motorcycle=1982
Number of samples from non-motorized_vehicle=1751
Number of samples from pedestrian=6262
Number of samples from pickup_truck=10000
Number of samples from single_unit_truck=5120
Number of samples from work_van=9679


In [4]:
objects = articulated_truck + bicycle + bus + car + motorcycle + non_motorized_vehicle + pedestrian + pickup_truck + single_unit_truck + work_van
bg_samples  = len(objects)
print(bg_samples)
background = load_folder('background', bg_samples)

67078
Number of samples from background=67078


In [8]:
# Function to compute HoG features of a list of images
# Adapted from Tutorial 4, Part 2
def compute_HoG(imgs, cell_size, block_size, nbins, h, w):
    img_features = []
    hog = cv2.HOGDescriptor(_winSize=((w // cell_size[1]) * cell_size[1],
                                  (h // cell_size[0]) * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)
    n_cells = (h // cell_size[0], w // cell_size[1])
    
    for img in imgs:

        hog_feats = hog.compute(img)\
                .reshape(n_cells[1] - block_size[1] + 1,
                        n_cells[0] - block_size[0] + 1,
                        block_size[0], block_size[1], nbins) \
               .transpose((1, 0, 2, 3, 4))  # index blocks by rows first
                
        # computation for BlockNorm

        gradients = np.full((n_cells[0], n_cells[1], nbins), 0, dtype=float)
        cell_count = np.full((n_cells[0], n_cells[1], 1), 0, dtype=int)

        for off_y in range(block_size[0]):
            for off_x in range(block_size[1]):
                gradients[off_y:n_cells[0] - block_size[0] + off_y + 1,
                          off_x:n_cells[1] - block_size[1] + off_x + 1] += \
                    hog_feats[:, :, off_y, off_x, :]
                cell_count[off_y:n_cells[0] - block_size[0] + off_y + 1,
                           off_x:n_cells[1] - block_size[1] + off_x + 1] += 1

        # Average gradients
        gradients /= cell_count
        
        img_features.append(gradients)
        
    return img_features

In [9]:
X = objects + background
y = ['object']*len(objects) + ['background']*len(background)

# Split the data into a training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=69)

# Define parameters for computing the HoG features
c_size = (4,4)
b_size = (4,4)
bins = 8
height = 64
width = 64

# Compute the HoG Features
HoG_feat = compute_HoG(X_train, c_size, b_size, bins, height, width)
HoG_feat = np.array(HoG_feat)

In [10]:
# Compute PCA for all components
pca = PCA(n_components = 2048)
X_PCA = pca.fit_transform(HoG_feat.reshape(HoG_feat.shape[0],-1))
# Find the variances explained
marginal_variances = pca.explained_variance_ratio_

total_variances = [sum(marginal_variances[:i]) for i in range(1,len(marginal_variances)+1)]

# Use the number of components representing 98% of the data
nb_components = next(x[0] for x in enumerate(total_variances) if x[1] > 0.98)
print('Number of components used: ' + str(nb_components))
pca = PCA(n_components = nb_components)
# Prepare training data
X_train_PCA = pca.fit_transform(HoG_feat.reshape(HoG_feat.shape[0],-1))
# Prepare test data
HoG_feat_test = compute_HoG(X_test, c_size, b_size, bins, height, width)
HoG_feat_test = np.array(HoG_feat_test)
X_test_PCA = pca.transform(HoG_feat_test.reshape(HoG_feat_test.shape[0],-1))

Number of components used: 518


In [11]:
# Fit the model
then = time.clock()
svc_clf = LinearSVC()
svc_clf.fit(X_train_PCA,y_train)
now = time.clock()
elapsed = now - then
print('Time to train LinearSVC = ' + str(elapsed))

Time to train LinearSVC = 18.826839335036233


In [12]:
# Predict the classes
then = time.clock()
svc_pred = svc_clf.predict(X_test_PCA)
now = time.clock()
elapsed = now - then
print('Time to predict using LinearSVC = ' + str(elapsed))
print('LinearSVC Prediction Results')
print(classification_report(y_test,svc_pred))

Time to predict using LinearSVC = 0.03180265962888029
LinearSVC Prediction Results
             precision    recall  f1-score   support

 background       0.91      0.91      0.91     10073
     object       0.91      0.91      0.91     10051

avg / total       0.91      0.91      0.91     20124



In [28]:
then = time.clock()
lr_clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
lr_clf.fit(X_train_PCA,y_train)
now = time.clock()
print(now - then)

13.294910474377048


In [29]:
then = time.clock()
lr_pred = lr_clf.predict(X_test_PCA)
now = time.clock()
print(now - then)
print(classification_report(y_test,lr_pred))

0.060136461610909464
             precision    recall  f1-score   support

 background       0.91      0.91      0.91     10073
     object       0.91      0.91      0.91     10051

avg / total       0.91      0.91      0.91     20124



## Localization Images

In [15]:
# Function to load the images given the folder name
def load_folder(num_samples):
    imgs = []
    files = listdir('./MIO-TCD-Localization/train/')
    img_counter = 0
    for img in files:
        # Read the images in grayscale
        img = cv2.cvtColor(cv2.imread('./MIO-TCD-Localization/train/{img}'.format(img=img)), cv2.COLOR_BGR2GRAY)
        # Perform a box blur on the images
        kernel = np.array([[1/9,1/9,1/9], [1/9,1/9,1/9], [1/9,1/9,1/9]])
        img = cv2.filter2D(img, -1, kernel)
        # Resize to reduce computation time
        imgs.append(img)
        img_counter += 1
        if(img_counter==num_samples): break
    return imgs


In [16]:
# The number of images from each class to import
num_samples = 100
# Import images from each folder
background_img = load_folder(num_samples)

In [ ]:
# resize factor for the image
resize_factor = 1/1.6

data_size = (64,64)

# get the dimensions of it to iterate through it
window_size = (72, 48)

windows = []

# iterate through list of images 
for i in range(len(background_img)):
    # get group photo in gray
    img_gray = background_img[i]
    
    windows.append([])
    grp_x, grp_y = img_gray.shape
    
    # iterate through the image and then resize the image each time
    #while(grp_x>window_size[0] and grp_y>window_size[1]):
    
    for y in range(0, grp_y-window_size[1], 20):
        for x in range(0, grp_x-window_size[0], 20):
            section = img_gray[x:x+window_size[0], y:y+window_size[1]]
            patch = cv2.resize(section, data_size)

            img = []
            img.append(patch)
            HoG = compute_HoG(img, c_size, b_size, bins, height, width)
            HoG = np.asarray(HoG)
            test_img = pca.transform(HoG.reshape(HoG.shape[0],-1))
            pred = lr_clf.predict(test_img)
            if(pred == 'object'):
                plt.imshow(patch)
                plt.show
                windows[i].append(patch)
                    
        #img_gray = cv2.resize(img_gray, (0,0), fx=resize_factor, fy=resize_factor) 
        #grp_x, grp_y = img_gray.shape

print(len(windows))